#### Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Testing the App Feedback Channel

This notebook shows you how to use the app feedback channel through the Android Management API. It is assumed that you have worked through the [Android Management API quickstart guide](https://developers.google.com/android/management/quickstart)  and are therefore familiar with how to use this colab notebook and already have a cloud project, test enterprise, and provisioned device.

## Authenticating

To create and access resources, you need to authenticate with an account that has edit rights over your project. To start the authentication flow, **run the cell below**.


In [0]:
import json
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pprint

# This is a public OAuth config that you can use to run this guide.
# However, use different credentials when building your own solution.
CLIENT_CONFIG = {
    'installed': {
        'client_id':'882252295571-uvkkfelq073vq73bbq9cmr0rn8bt80ee.apps.googleusercontent.com',
        'client_secret': 'S2QcoBe0jxNLUoqnpeksCLxI',
        'auth_uri':'https://accounts.google.com/o/oauth2/auth',
        'token_uri':'https://accounts.google.com/o/oauth2/token'
    }
}
SCOPES = ['https://www.googleapis.com/auth/androidmanagement']

# Run the OAuth flow.
flow = InstalledAppFlow.from_client_config(CLIENT_CONFIG, SCOPES)
credentials = flow.run_console()

# Create the API client.
androidmanagement = build('androidmanagement', 'v1', credentials=credentials)

print('\nAuthentication succeeded.')

## Declare your enterprise
If you already have an enterprise, you can enter the enterprise name in the cell below and run the cell. If you do not have a test enterprise, please go through the [general Quickstart](https://developers.google.com/android/management/quickstart) 

In [0]:
# Paste your enterprise name here.
# It should be of the format 'enterprises/LC...'
enterprise_name = ''

In [0]:
# Enter the package name for the application that you want to test app feedback for here.
app_name = ''


Below is an example of how to create or update a policy for apps on a device with settings controlling the behavior of status reports. To enable reporting for a device, update the device policy and set StatusReportingSettings.applicationReportsEnabled to true.



In [0]:
# Example policy below.
# Please specify the type of installation to perform for an app.

policy_name = enterprise_name + '/policies/feedback-test'

policy_json = '''
{
  "applications": [
    {
      "packageName": "''' + app_name + '''",
      "installType": "FORCE_INSTALLED"
    }
  ],
  "debuggingFeaturesAllowed": true,
  "statusReportingSettings" : {
    "applicationReportsEnabled": true,
    "deviceSettingsEnabled" : true
    }
}
'''

androidmanagement.enterprises().policies().patch(
    name=policy_name,
    body=json.loads(policy_json)
).execute()

In [0]:
# List devices so you can get a device name to use to pull app feedback below.
androidmanagement.enterprises().devices().list(
    parent=enterprise_name
).execute()

To review a devices's latest report at any time, call devices.get() 

IT admins can use the data from keyed app states to retrieve information about device states.

In [0]:
# Enter the device Id
device_Id = ''

In [0]:
# Review a device’s latest report:
androidmanagement.enterprises().devices().get(
    name=enterprise_name + '/devices/' + device_Id
).execute()

#   Set up [Pub/Sub notifications](https://developers.google.com/android/management/notifications)

As an alternative you can set up Pub/Sub notifications to provide a way for you to receive alerts about newly enrolled devices, device reports, and recently issued commands

In [0]:
# Go to your Cloud Console to get your project ID.
# Enter your Google Cloud Project ID below:
cloud_project_id = ''

In [0]:
# Go to the Pub/Sub topics page in the Cloud Console to get the topic name you created.
# Enter the topic you created below:
my_topic = ''

In [0]:

# Update the enterprise to support notifications:
enterprise = androidmanagement.enterprises().patch(
    name=enterprise_name,
    body={
        "name" : enterprise_name,
        "enabledNotificationTypes" : "STATUS_REPORT",
        "pubsubTopic": "projects/" + cloud_project_id + "/topics/" + my_topic
    }
).execute()